In [115]:
#AI Trainer for the Adjustments Project


In [116]:
#Load input data
import pandas as pd
import numpy as np

#Load data
df=pd.read_csv('SampleData.csv')
df.head(10)

,AdjustmentType,AdjustmentName,AdjustedField,Drawn,Undrawn,PD,CustomerID,CustomerType,RiskCategory,IsDefaulted,CountryCode,NewVal
0,Amend,Fix_Broken_Batch,RiskCategory,706413,586038,0.45,CUST238947,SME,Low,1,US,Low
1,Amend,Fix_Broken_Batch,IsDefaulted,573613,902614,0.11,CUST981372,Corporate,Low,0,DE,1
2,Amend,Default,CustomerID,466772,38940,0.87,CUST103289,Corporate,Low,0,GB,CUST712035
3,Amend,Defaulting_Reclassify,CustomerType,288555,765762,0.24,CUST238947,SME,Medium,0,DE,Retail
4,Amend,Credit_Uplift,CustomerType,694503,686516,0.71,CUST712035,SME,High,0,US,Corporate
5,Amend,Default,CustomerType,471937,680522,0.80,CUST238947,Corporate,High,1,GB,SME
6,Amend,Default,CustomerType,758318,47670,0.50,CUST563201,Corporate,High,0,US,Corporate
7,Amend,Credit_Uplift,CustomerID,332147,203342,0.71,CUST359147,SME,Low,1,US,CUST665428
8,Amend,Fix_Broken_Batch,CustomerID,152030,387137,0.84,CUST359147,Corporate,Medium,1,GB,CUST192654
9,Amend,Fix_Broken_Batch,CountryCode,405353,128059,0.03,CUST665428,SME,Low,0,DE,DE


In [121]:
#Stage 1: OneHot encode string labels so we can use them in the model

#Build list of all string columns in the dataframe
string_columns=[]
for col in df.columns:
    if df[col].dtype=='object':
        string_columns.append(col)

#Turn output labels into numbers
outputLabels=df["NewVal"].unique()
# #Create a dictionary to map the output labels to numbers
outputLabelsDict={}
for i in range(len(outputLabels)):
    outputLabelsDict[outputLabels[i]]=i
df["NewVal"]=df["NewVal"].map(outputLabelsDict)

df_encoded=pd.get_dummies(df, columns=string_columns)

#Move NewVal to the end of the dataframe
df_encoded=df_encoded[[c for c in df_encoded if c not in ['NewVal']] + ['NewVal']]

df_encoded


In [108]:
#Stage 2: Normalize all numeric values
#Normalize the dataframe

#Normalize Drawn (-1 to 1)
df_normalized=df_encoded.copy()
df_normalized["Drawn"]=(df_encoded["Drawn"]-df_encoded["Drawn"].min())/(df_encoded["Drawn"].max()-df_encoded["Drawn"].min())*2-1
#Normalize Undrawn (-1 to 1)
df_normalized["Undrawn"]=(df_encoded["Undrawn"]-df_encoded["Undrawn"].min())/(df_encoded["Undrawn"].max()-df_encoded["Undrawn"].min())*2-1

#Clip PD to 0-1
df_normalized["PD"]=df_normalized["PD"].clip(0,1)


df_normalized

,Drawn,Undrawn,PD,IsDefaulted,AdjustmentType_Amend,AdjustmentName_Credit_Uplift,AdjustmentName_Default,AdjustmentName_Defaulting_Reclassify,AdjustmentName_Fix_Broken_Batch,AdjustmentName_GRC_EndMonth,...,CustomerType_Corporate,CustomerType_Retail,CustomerType_SME,RiskCategory_High,RiskCategory_Low,RiskCategory_Medium,CountryCode_DE,CountryCode_GB,CountryCode_US,NewVal
0,0.457108,0.497036,0.12,1,1,0,0,0,0,1,...,0,0,1,0,0,1,0,1,0,0
1,0.237189,-0.206909,0.71,1,1,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,1
2,-0.951375,0.961725,0.49,0,1,0,0,0,1,0,...,1,0,0,0,0,1,0,0,1,2
3,-0.565550,-0.942394,0.54,0,1,0,0,0,1,0,...,1,0,0,0,1,0,1,0,0,3
4,0.905760,0.602994,0.50,1,1,0,0,0,0,1,...,0,0,1,0,0,1,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-0.363267,-0.053124,0.94,1,1,0,0,0,1,0,...,1,0,0,0,1,0,1,0,0,15
996,0.576710,-0.949010,0.61,0,1,0,0,0,0,1,...,0,0,1,1,0,0,0,1,0,10
997,-0.857825,-0.694044,0.27,1,1,0,0,0,0,1,...,1,0,0,1,0,0,1,0,0,11
998,-0.970167,0.019310,0.30,1,1,0,0,0,0,1,...,0,0,1,0,1,0,1,0,0,6


In [31]:
# #Split the data into test and training sets using pandas
# msk = np.random.rand(len(df_normalized)) < 0.8
# train = df_normalized[msk]
# test = df_normalized[~msk]


In [85]:
#Turn the pandas dataframe into a numpy array, split out the label and then break into sets
import tensorflow as tf

#turn df_normalized into a normal array
data=df_normalized.values

#Get all columns except the output column
X=data[:,1:]
#Get the output column
y=data[:,-1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [87]:
X_train.shape[1]

34

In [109]:
#Time to build the model

#Simple feed-forward model. No U-nets or anything fancy
ann = tf.keras.models.Sequential()

neurons=1000

#Input and first hidden layer
ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))
ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))
ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))

#Output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#Compile!
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



In [110]:
#Train the MODEL!!!!!
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
25/25 [==============================] - 1s 18ms/step - loss: -5961.3193 - accuracy: 0.0725
Epoch 2/100
25/25 [==============================] - 0s 19ms/step - loss: -179312.1875 - accuracy: 0.0737
Epoch 3/100
25/25 [==============================] - 0s 19ms/step - loss: -1624657.6250 - accuracy: 0.0737
Epoch 4/100
25/25 [==============================] - 0s 19ms/step - loss: -8252153.5000 - accuracy: 0.0737
Epoch 5/100
25/25 [==============================] - 0s 18ms/step - loss: -29366812.0000 - accuracy: 0.0737
Epoch 6/100
25/25 [==============================] - 0s 18ms/step - loss: -80855392.0000 - accuracy: 0.0737
Epoch 7/100
25/25 [==============================] - 0s 18ms/step - loss: -185724608.0000 - accuracy: 0.0737
Epoch 8/100
25/25 [==============================] - 0s 19ms/step - loss: -379658720.0000 - accuracy: 0.0737
Epoch 9/100
25/25 [==============================] - 0s 19ms/step - loss: -690334528.0000 - accuracy: 0.0737
Epoch 10/100
25/25 [=============